## Importing libraries and files

In [1]:
import pandas as pd
from keras.models import load_model
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


ann_arr = load_model('RP_Arrival_time/Arrival_ANN1')
ann_end = load_model('RP_End_time/End_ANN1')
ann_chg = load_model('RP_Change_time/Change_ANN1')
ann_trg = load_model('RP_Trough_time/Trough_ANN1')

#Section1
#ann1 = load_model('Pressure_profiles_codes_section_1_regression/ANN_13.h5')
#sc1 = pickle.load(open('Pressure_profiles_codes_section_1_regression/scaler_ANN.pkl', 'rb'))
#Section2
#ann2 = load_model('Pressure_profiles_codes_section_2_regression/ANN_1.h5')
#sc2 = pickle.load(open('Pressure_profiles_codes_section_2_regression/scaler_ANN.pkl', 'rb'))
#Section3
ann3 = load_model('RP_Section_3_new/Section3_new_ANN_1')
sc3 = pickle.load(open('RP_Section_3_new/Section3_new_scaler_ANN_1.pkl', 'rb'))
#Section4
#ann4 = load_model('Pressure_profiles_codes_section_4_regression/ANN_1.h5')
#sc4 = pickle.load(open('Pressure_profiles_codes_section_4_regression/scaler_ANN.pkl', 'rb'))
#Section5
#ann5 = load_model('Pressure_profiles_codes_section_5_regression/ANN_2.h5')
#sc5 = pickle.load(open('Pressure_profiles_codes_section_5_regression/scaler_ANN2.pkl', 'rb'))

## Input details

In [7]:
Mass = 16.9
Standoff_distance = 10.3
Angle = 10
termination_time = 35
interval = 0.1

## Pressure Profile

In [8]:
arr = np.array ([[Mass,Standoff_distance, Angle]])

arrival_time = ann_arr.predict(arr)
end_time = ann_end.predict(arr)
change_time = ann_chg.predict(arr)
trough_time = ann_trg.predict(arr)

print ("arrival", arrival_time, "end", end_time, "change", change_time, "trough", trough_time)

def stable_state(start, end, interval):    
    t = np.arange(start, end + interval, interval)
    t = t.reshape(len(t),1)

    p = np.repeat(0, len(t)).reshape(len(t),1)
    
    values = np.concatenate ((t, p), axis = 1)
    return values

'''def p_fluctuation(standoff_distance, d_set):
    if 3 <= standoff_distance <= 7:
        d_set[:,0:3] = sc1.transform(d_set[:,0:3])
        p_f = ann1.predict(d_set)
        
    elif 7 < standoff_distance <= 13:
        d_set[:,0:3] = sc2.transform(d_set[:,0:3])
        p_f = ann2.predict(d_set)
        
    elif 13 < standoff_distance <= 19:
        d_set[:,0:3] = sc3.transform(d_set[:,0:3])
        p_f = ann3.predict(d_set)
        
    elif 19 < standoff_distance <= 25:
        d_set[:,0:3] = sc4.transform(d_set[:,0:3])
        p_f = ann4.predict(d_set)
        
    elif 25 < standoff_distance <= 30:
        d_set[:,0:3] = sc5.transform(d_set[:,0:3])
        p_f = ann5.predict(d_set)
        
    else:
        print ("Out of trained range")
    return p_f'''

def p_fluctuation2(d_set):
    d_set[:,0:4] = sc3.transform(d_set[:,0:4])
    p_f = ann3.predict(d_set)
    
    return p_f

if termination_time < arrival_time:
    start = 0
    end = termination_time
    values1 = stable_state(start, end, interval)
    arrival_time = 0
    end_time = 0

else:
    start = 0
    end = arrival_time
    values1 = stable_state(start, end, interval)
    
    if termination_time < end_time:
        end_time = termination_time
        
    else:
        start = end_time
        end = termination_time
        values3 = stable_state(start, end, interval)

t2= np.arange(arrival_time, end_time + interval, interval)
t2 = t2.reshape(len(t2),1)

if t2[0] < change_time:
    ph2 = np.array([[0, 0, 1]])
elif change_time <= t2[0] < trough_time:
    ph2 = np.array([[0, 1, 0]])
else:
    ph2 = np.array([[1, 0, 0]])
    
positive = np.array([0, 0, 1])
negative_increasing = np.array([0, 1, 0])
negative_decreasing = np.array([1, 0, 0])

for time in t2[1:]:
    if time < change_time:
        ph2=np.vstack((ph2,positive))
    elif change_time <= time < trough_time:
        ph2=np.vstack((ph2,negative_increasing))
    else:
        ph2=np.vstack((ph2,negative_decreasing))
        
m2 = np.repeat(Mass, len(t2)).reshape(len(t2),1)
d2 = np.repeat(Standoff_distance, len(t2)).reshape(len(t2),1)
a2 = np.repeat(Angle, len(t2)).reshape(len(t2),1)
d_set = np.concatenate ((m2, d2, a2, t2, ph2), axis = 1)

#p2 = p_fluctuation(Standoff_distance, d_set)
p2 = p_fluctuation2(d_set)
p2 = p2.reshape((-1, 1))
values2 = np.concatenate ((t2, p2), axis = 1)


if 'values2' in globals():
    ml_model = np.concatenate((values1, values2), axis=0)
    if 'values3' in globals():
        ml_model = np.concatenate((ml_model, values3), axis=0)
else:
    ml_model = values1

1/1 [==============================] - 0s 47ms/step
arrival [[17.320383]] end [[55.518246]] change [[25.602268]] trough [[28.889635]]
6/6 [==============================] - 0s 0s/step


C:\Users\z5379606\AppData\Local\Temp\ipykernel_27912\3828193549.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(start, end + interval, interval)
C:\Users\z5379606\AppData\Local\Temp\ipykernel_27912\3828193549.py:70: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t2= np.arange(arrival_time, end_time + interval, interval)


In [9]:
file_path = Path(f'G:/Chamodi/LSDYNA3D/RP_TH_models/RP_Comparison_with_Numerical/10.3m16.9kg/2.xlsx')
lsdyna = pd.read_excel(file_path)
X2 = lsdyna['Time']
y2 = lsdyna['Pressure']

#print (standoff_distance)
#print ("ML max", max(y1))
#print ("LSDYNA max", max(y2))

In [12]:
X1 = ml_model[:,0]
y1 = ml_model[:,1]


plt.figure(figsize=(9,6))
plt.plot(X1, y1, linestyle = 'dashed', color = 'red', label = 'ANN based system')
plt.plot(X2, y2, color = 'blue',label = 'LS-DYNA')
plt.ylabel('Incident Pressure (kPa)',fontsize = 18)
plt.xlabel('Time (ms)',fontsize = 18)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.legend(fontsize = 18)
plt.xlim (15,35)
plt.savefig('Section3_ANN1.png', dpi=300, bbox_inches='tight')
plt.show()